In [1]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [2]:
def setDocumentContent():
    with open('/Users/sameernawaz/AnacondaProjects/Jupyter Notebook/ML Personalisation/Datasets/trainingTextClassification.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        count = 0
        for line in csv_reader:
            val[count] = line
            count = count + 1
            if count >= sizeOfSets :
                break
            
    countTemp = 0
    for i in val:
        document[countTemp] = i[5]
        countTemp += 1

In [3]:
sizeOfSets = 1000
val = [None] * sizeOfSets
document = [""] * sizeOfSets

setDocumentContent()

In [4]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
print(val[0])

['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY', '_TheSpecialOne_', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]


In [7]:
df = pd.DataFrame(val, columns = ['Inf', 'ID', 'Time', 'Query', 'Username', 'Tweet']) 

In [8]:
df.head()

,Inf,ID,Time,Query,Username,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
df = df[['ID', 'Username', 'Tweet']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 3 columns):
ID          1000 non-null object
Username    1000 non-null object
Tweet       1000 non-null object
dtypes: object(3)
memory usage: 31.2+ KB


In [11]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['Tweet']))
len(norm_corpus)

1000

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(1000, 1071)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.358548,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.135813,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
movies_list = df['ID'].values
movies_list, movies_list.shape

(array(['1467810369', '1467810672', '1467810917', '1467811184',
        '1467811193', '1467811372', '1467811592', '1467811594',
        '1467811795', '1467812025', '1467812416', '1467812579',
        '1467812723', '1467812771', '1467812784', '1467812799',
        '1467812964', '1467813137', '1467813579', '1467813782',
        '1467813985', '1467813992', '1467814119', '1467814180',
        '1467814192', '1467814438', '1467814783', '1467814883',
        '1467815199', '1467815753', '1467815923', '1467815924',
        '1467815988', '1467816149', '1467816665', '1467816749',
        '1467817225', '1467817374', '1467817502', '1467818007',
        '1467818020', '1467818481', '1467818603', '1467818900',
        '1467819022', '1467819650', '1467819712', '1467819812',
        '1467820206', '1467820835', '1467820863', '1467820906',
        '1467821085', '1467821338', '1467821455', '1467821715',
        '1467822384', '1467822389', '1467822519', '1467822522',
        '1467822687', '1467822918', '146

In [100]:
movie_idx = np.where(movies_list == '1467821338')[0][0]
movie_idx

53

In [107]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [102]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([836, 105, 262, 449, 221])

In [103]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['1468018610', '1467837602', '1467878057', '1467927016',
       '1467871226'], dtype=object)

In [108]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [110]:
popular_movies = ['1468021282', '1468054887', '1468052499', '1468053869', '1468055266']

In [111]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))

Movie: 1468021282
Top 5 recommended Movies: ['1467907298' '1467990374' '1467982442' '1467951016' '1467918812']
Movie: 1468054887
Top 5 recommended Movies: ['1467877865' '1467931396' '1467835880' '1467861095' '1467892945']
Movie: 1468052499
Top 5 recommended Movies: ['1467980251' '1467980262' '1467980679' '1467980858' '1467981007']
Movie: 1468053869
Top 5 recommended Movies: ['1467896463' '1467852067' '1467924273' '1467932979' '1468011315']
Movie: 1468055266
Top 5 recommended Movies: ['1467844907' '1468023065' '1468033152' '1467842568' '1468001960']


In [115]:
print((df.loc[df['ID'] == "1468054887"]['Tweet']).values[0])
print('\n')
for i in movie_recommender(movie_title = "1468054887", movies=movies_list, doc_sims=doc_sim_df):
    print("=>",(df.loc[df['ID'] == i]['Tweet']).values[0])

There's an inch of snow on the ground (and counting). I'm worried about the poor flowers 


=> @kissability Me too I is poor 
=> http://twitpic.com/2y36e - cant see the flowers falling  i dont have a camera, just my cellphone
=> Poor cameron (the hills) 
=> too worried and tired to post tonight 
=> Poor Joshy is sick???   those damn tejanos!
